# 3.1 RecyclerView

Aplikację rozpoczniemy od utworzenia layoutu pojedynczego elementu listy. Przechodzimy do **res -> layout** i z menu kontekstowego wybieramy **New -> Layout Resource File** i jako nazwę podaję `word_list_item`. Przejdźmy do nowoutworzonego pliku i dodajmy pole `TextView` w którym będziemy wyświetlać słowo.

```xml
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout
    xmlns:android="http://schemas.android.com/apk/res/android"
    android:layout_width="match_parent"
    android:orientation="horizontal"
    android:layout_height="wrap_content">

    <TextView
        android:id="@+id/singleWord"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:textSize="24sp"/>

</LinearLayout>
```

Przejdźmy do pliku `MainActivity.java` i utwórzmy naszą listę słów

```java
public class MainActivity extends AppCompatActivity {

    private final LinkedList<String> wordList = new LinkedList<>();
    
    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        setContentView(R.layout.activity_main);
        
        for (int i = 0; i < 30; i++)
            wordList.add("Word" + i);
    }
}
```

Następnie utworzymy klasę `WordListAdapter.java`

```java
public class WordListAdapter extends RecyclerView.Adapter<WordListAdapter.WordListViewHolder> {
}
```

Klasa musi rozszerzać zagnieżdżoną klasę abstrakcyjną `Adapter` klasy `RecyclerView`. Musimy podać parametr typowany - jest to klasa rozszerzająca `ViewHolder` która będzie wykorzystana przez `Adapter`. Dodajemy ją jako klasę wewnętrzną

```java
public class WordListAdapter extends RecyclerView.Adapter<WordListAdapter.WordListViewHolder> {

    protected class WordListViewHolder extends RecyclerView.ViewHolder {
        public WordListViewHolder(@NonNull View itemView) {
            super(itemView);
        }
    }
}
```

Utworzona klasa wewnętrzna `WordListViewHolder` musi rozszerzać zagnieżdżoną abstrakcyjną klasę `ViewHolder` klasy `RecyclerView`. Ponieważ klasa `WordListAdapter` rozszerza klasę `Adapter` więc musi implementować wszystkie jej metody.

```java
public class WordListAdapter extends RecyclerView.Adapter<WordListAdapter.WordListViewHolder> {
    @NonNull
    @Override
    public WordListAdapter.WordListViewHolder onCreateViewHolder(@NonNull ViewGroup parent, int viewType) {
        return null;
    }

    @Override
    public void onBindViewHolder(@NonNull WordListAdapter.WordListViewHolder holder, int position) {

    }

    @Override
    public int getItemCount() {
        return 0;
    }

    protected class WordListViewHolder extends RecyclerView.ViewHolder {
        public WordListViewHolder(@NonNull View itemView) {
            super(itemView);
        }
    }
}
```

Nasz layout elementu listy może być bardzo skomplikowany i zawierać wiele elementów - w tym pprostym przykładzie mamy tylko jedno pole `TextView`. W przypadku gdy mamy wiele elementów i chcemy wykonać jakąś akcję - ale na całym elemencie listy a nie na jego części, musimy mieć możliwość zdefiniowania wspólnej metody dla całego elementu. Klasa `ViewHolder` udostępnia `itemView`, który oferuje taką funkcjonalność - do całego elementu możemy się odnieść poprzez `itemView`.

Implementację rozpocznijmy od połączenia layoutu elementu z `ViewHolder`. w klasie `WordListViewHolder` dodaję pole `TextView` i w konstruktorze wykon8ujemy przypisanie przez wywołanie metody `findViewById`.

```java
    protected class WordListViewHolder extends RecyclerView.ViewHolder {

        private final TextView wordText;

        public WordListViewHolder(@NonNull View itemView) {
            super(itemView);
            wordText = itemView.findViewById(R.id.singleWord);
        }
    }
```

Zwróćmy uwagę że metodę `findViewById` wykonujemy na `itemView` - tak postępujemy ze wszystkimi elementami layoutu. Drugim polem które będziemy potrzebować jest referencja do `WordListAdapter`, którego instancję przekażemy w konstruktorze `WordListViewHolder`.

```java
    protected class WordListViewHolder extends RecyclerView.ViewHolder {

        private final TextView wordText;
        final WordListAdapter adapter;

        public WordListViewHolder(@NonNull View itemView, WordListAdapter adapter) {
            super(itemView);
            this.adapter = adapter;
            wordText = itemView.findViewById(R.id.singleWord);
        }
    }
```

Przjdźmy do klasy `WordListAdapter`, dodajmy listę ze słowami i konstruktor.

```java
    private LinkedList<String> wordList;

    public WordListAdapter(Context context, LinkedList<String> wordList){
        this.wordList = wordList;
    }
```

Drugim polem które będzie nam potrzebne jest `LayoutInflater`

```java
    private LinkedList<String> wordList;
    private final LayoutInflater inflater;

    public WordListAdapter(Context context, LinkedList<String> wordList){
        inflater = LayoutInflater.from(context);
        this.wordList = wordList;
    }
```

`LayoutInflater` tworzy obiekty `View` na podstawie pliku `xml` ze zdefiniowanym layoutem. Instancję `LayoutInflater` otrzymujemy wywołując metodę `from` i przekazując do niej `Context`. `Context` zawiera informacje o cały środowisku naszej aplikacji i o jej aktualnym stanie. Dzięki tej klasie (implementowanej automatycznie) możemy uzyskać dostęp do zasobów naszej aplikacji, mamy możliwość interakcji pomiędzy różnymi komponentami aplikacji

Rozpocznijmy implementację wszystkich metod. Rozpoczniemy od `getItemCount` - metoda zwraca ilość elementów. Tutaj będzie to rozmiar listy

```java
    @Override
    public int getItemCount() {
        return wordList.size();
    }
```

Przejdźmy do metody `onCreateViewHolder`. W pierwszym kroku wywołamy metodę `inflate` na obiekcie `LayoutInflater`, tworzy ona hierarchię obiektów na podstawie pliku `xml`.

```java
View itemView = inflater.inflate(R.layout.word_list_item, parent, false);
```

Przyjmuje trzy argumenty
- `resource` - wskazujemy nazwę zasobu ze zdefiniowanym layoutem
- `ViewGroup` - rodzic utworzonej hierarchii
- `attachToRoot` - dodanie utworzonej hierarchii do rodzica - jeżeli `true` dodanie jest natychmiastowe - najczęściej ustawiamy na `false`.

Metoda zwraca `ViewHolder`, więc wywołujemy konstruktor klasy `WordListViewHolder`

```java
    @NonNull
    @Override
    public WordListAdapter.WordListViewHolder onCreateViewHolder(@NonNull ViewGroup parent, int viewType) {
        View itemView = inflater.inflate(R.layout.word_list_item, parent, false);
        return new WordListViewHolder(itemView, this);
    }
```

Ostatnią metodą jest `onBindViewHolder`, tutaj połączymy dane z odpowiednimi elementami layoutu

```java
    @Override
    public void onBindViewHolder(@NonNull WordListAdapter.WordListViewHolder holder, int position) {
        String current = wordList.get(position);
        holder.wordText.setText(current);
    }
```

Przy tworzeniu `RecylcerView` następuje automatyczna iteracja po wszystkich elementach listy. Przy każdej iteracji wywoływana jest metoda `onBindViewHolder`. Więc przy każdej iteracji wyciągamy tekst z `LinkedList` o zadanym indeksie, który jest taki sam jak pozycja na liście więc używamy `position` i ustawiamy tekst pola `TextView` do którego dostajemy się poprzez przekazany w argumencie metody `WordListViewHolder`.

Pełny kod klasy `WordListAdapter`

```java
package pl.edu.uwr.pum.recyclerviewwordlistjava;

import android.content.Context;
import android.view.LayoutInflater;
import android.view.View;
import android.view.ViewGroup;
import android.widget.TextView;

import androidx.annotation.NonNull;
import androidx.recyclerview.widget.RecyclerView;

import java.util.LinkedList;

public class WordListAdapter extends RecyclerView.Adapter<WordListAdapter.WordListViewHolder> {

    private LinkedList<String> wordList;
    private final LayoutInflater inflater;

    public WordListAdapter(Context context, LinkedList<String> wordList) {
        inflater = LayoutInflater.from(context);
        this.wordList = wordList;
    }

    @NonNull
    @Override
    public WordListAdapter.WordListViewHolder onCreateViewHolder(@NonNull ViewGroup parent, int viewType) {
        View itemView = inflater.inflate(R.layout.word_list_item, parent, false);
        return new WordListViewHolder(itemView, this);
    }

    @Override
    public void onBindViewHolder(@NonNull WordListAdapter.WordListViewHolder holder, int position) {
        String current = wordList.get(position);
        holder.wordText.setText(current);
    }

    @Override
    public int getItemCount() {
        return wordList.size();
    }

    protected static class WordListViewHolder extends RecyclerView.ViewHolder {

        private final TextView wordText;
        final WordListAdapter adapter;

        public WordListViewHolder(@NonNull View itemView, WordListAdapter adapter) {
            super(itemView);
            this.adapter = adapter;
            wordText = itemView.findViewById(R.id.singleWord);
        }
    }
}
```

Dodajmy `RecyclerView` do `MainActivity`, w pierwszym kroku zmodyfikujemy layout.

```xml
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    tools:context=".MainActivity">

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/recyclerView"
        android:layout_width="match_parent"
        android:layout_height="wrap_content" />

</LinearLayout>
```

Następnie w klasie `MainActivity.java` dodajemy `RecyclerView` i łączymy go z elementem layoutu.

```java
RecyclerView recyclerView = findViewById(R.id.recyclerView);
```

Teraz musimy utworzyć `WordListAdapter` i wskazać go jako adapter utworzonego `RecyclerView` przez wywołanie metody `setAdapter`.

```java
WordListAdapter wordListAdapter = new WordListAdapter(this, wordList);
recyclerView.setAdapter(wordListAdapter);
```

Jako `Context` przekazujemy `this` ponieważ wszystkie klasy `Activity` rozszerzają klasę `Context`. Ostatnim elementem jest ostawienie `LayoutManager`, tutaj będzie to `LinearLayoutManager` - ustawia element zarządzający layoutem tego konkretnego `RecyclerView`.

```java
recyclerView.setLayoutManager(new LinearLayoutManager(this));
```

Pełny kod klasy `MainActivity`

```java
package pl.edu.uwr.pum.recyclerviewwordlistjava;

import androidx.appcompat.app.AppCompatActivity;
import androidx.recyclerview.widget.LinearLayoutManager;
import androidx.recyclerview.widget.RecyclerView;

import android.os.Bundle;

import java.util.LinkedList;
import java.util.concurrent.LinkedBlockingDeque;

public class MainActivity extends AppCompatActivity {

    private final LinkedList<String> wordList = new LinkedList<>();

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        setContentView(R.layout.activity_main);

        for (int i = 0; i < 50; i++)
            wordList.addLast("Word" + i);

        RecyclerView recyclerView = findViewById(R.id.recyclerView);
        WordListAdapter wordListAdapter = new WordListAdapter(this, wordList);
        recyclerView.setAdapter(wordListAdapter);
        recyclerView.setLayoutManager(new LinearLayoutManager(this));
    }
}
```

Na tym etapie możemy zbudować i przetestować naszą aplikację.

### Dodanie obsługi `onClick` dla każdego elementu listy

Kolejnym etapem będzie dodanie wspólnej obsługi metody `onClick`. W tym celu przechodzimy do klasy `WordListAdapter.java` i w konstruktorze klasy `WordListViewHolder` dodajemy obsługę `onClick` na `itemView`

```java
        public WordListViewHolder(@NonNull View itemView, WordListAdapter adapter) {
            super(itemView);
            this.adapter = adapter;
            wordText = itemView.findViewById(R.id.singleWord);
            
            itemView.setOnClickListener(view -> {
                int position = getLayoutPosition();
                String element = wordList.get(position);
                wordList.set(position, "Clicked!" + element);
                adapter.notifyItemChanged(position);
            });
        }
```

Pozycję klikniętego elementu dostajemy przez wywołanie `getLayoutPosition`, modyfikujemy element `LinkedList` i wywołujemy metodę `notifyItemChanged` na adapterze w celu powiadomienia o zmianie. Po otrzymaniu adapter automatycznie odświeży element o zadanej pozycji.

Teraz po każdym kliknięciu na element na liście zostanie zmodyfikowany wyświetlany napis.